In [ ]:
# !pip install SMOTE

In [ ]:
# !pip install imbalanced-learn

In [ ]:
# !pip install -U scikit-learn imbalanced-learn

In [ ]:
# pip install pyldavis

In [ ]:
# !pip install tensorflow transformers

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis.lda_model
import tensorflow as tf
from transformers import TFBertModel, BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.optimizers import legacy


/opt/homebrew/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/homebrew/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [2]:
from functions import *

In [3]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [ ]:
train.head(20)

In [ ]:
train.isnull().sum()

In [ ]:
# train = train.dropna()

In [ ]:
train.isnull().sum()

In [ ]:
train.duplicated().sum()

In [ ]:
train = train.drop_duplicates()

In [ ]:
train.shape

In [ ]:
class_counts = train['overall'].value_counts()

plt.bar(class_counts.index, class_counts.values)
plt.xlabel('Class')
plt.ylabel('Number of Samples')
plt.title('Histogram of Sample Counts per Class')
plt.show()

In [ ]:
test.describe()

In [ ]:
test.head(20)

In [ ]:
test.isnull().sum()

In [ ]:
test.duplicated().sum()

In [ ]:
test.shape

In [4]:
train["Review"]= train["Review"].astype(str)


In [5]:
train["Review"] = train["Review"].apply(preprocess)

KeyboardInterrupt: 

In [7]:
train

,overall,Review
0,5,I love these glitter pens. They sparkle deligh...
1,5,It works well with my machine. I use mostly c...
2,5,"This is a great assortment of colors, though t..."
3,5,Just what I was looking for.
4,5,I make 400 birds for the hospital each month.
...,...,...
370858,5,I love these dies they make great backgrounds ...
370859,5,I love Darice embossing folders. Darcie's fol...
370860,5,I ordered these to add to my Earthy Markers th...
370861,4,Made perfect off white color by blending a tin...


In [5]:
X = train["Review"]
y = train["overall"]
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y,test_size=0.3, random_state=42)
print('Original class distribution:', Counter(y_train))

Original class distribution: Counter({5: 198216, 4: 32082, 3: 15029, 1: 7540, 2: 6737})


In [1]:
# load tokenizer and model
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=5)

NameError: name 'BertTokenizer' is not defined

In [7]:
reviews = X_train.tolist()
labels = y_train

In [8]:
# to start from 0
labels = [label - 1 for label in labels]

In [9]:
# encode texts
encodings = tokenizer(reviews, padding=True, truncation=True, return_tensors='tf')

In [10]:
# convert labels to one-hot
# 4 -> cuz 5 labels and 0 to 4 is the range
one_hot_labels = tf.keras.utils.to_categorical(labels, num_classes=5)

In [26]:
print(len(encodings['input_ids']))
print(len(one_hot_labels))          


259604
259604


In [11]:
total_size = len(reviews)
train_size = int(total_size * 0.8)  #80-20 split
test_size = total_size - train_size

In [12]:
# create a TensorFlow dataset
dataset = tf.data.Dataset.from_tensor_slices((dict(encodings), one_hot_labels))
dataset = dataset.shuffle(len(reviews)).batch(15)

In [13]:
# Shuffle the dataset
shuffled_dataset = dataset.shuffle(buffer_size=total_size, reshuffle_each_iteration=False)

In [14]:
model.compile(optimizer=legacy.Adam(learning_rate=5e-5), loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
model.fit(shuffled_dataset, epochs=1, verbose=1)

   53/17307 [..............................] - ETA: 148:29:22 - loss: 10.8965 - accuracy: 0.1459

KeyboardInterrupt: 

In [ ]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
y_train_pred = model.predict(X_train)
print(classification_report(y_train, y_train_pred))

In [ ]:
high_rated = train[train['overall'] == 5]
low_rated = train[train['overall'] == 1]

In [ ]:
high_rated['Review'] = high_rated['Review'].apply(preprocess)
low_rated['Review'] = low_rated['Review'].apply(preprocess)

In [ ]:
high = vectorizer.fit_transform(high_rated['Review'])
low = vectorizer.fit_transform(low_rated['Review'])

In [ ]:
# Number of topics
n_topics = 10

# Run LDA
lda_high = LatentDirichletAllocation(n_components=n_topics, random_state=0)
lda_high.fit(high)

lda_low = LatentDirichletAllocation(n_components=n_topics, random_state=0)
lda_low.fit(low)



In [ ]:
panel_high = pyLDAvis.lda_model.prepare(lda_high, high, vectorizer, mds='tsne')
pyLDAvis.display(panel_high)

In [ ]:
# Visualize low-rated topics
panel_low = pyLDAvis.lda_model.prepare(lda_low, low, vectorizer, mds='tsne')
pyLDAvis.display(panel_low)